In [ ]:
import multiprocessing
import threading
from tqdm import tqdm
from niapy.algorithms.basic import (
    BatAlgorithm,
    FireflyAlgorithm,
    ParticleSwarmOptimization,
)
from niapy.algorithms import Algorithm
from niapy.problems import Problem
from niapy.problems.schwefel import Schwefel
from niapy.task import Task
import torch
from torch import nn
from torch.utils import data
from PIL import Image
import pandas as pd
import seaborn as sb
import sklearn.model_selection
from sklearn.decomposition import PCA
from sklearn.metrics import confusion_matrix, classification_report
from matplotlib import pyplot as plt
import os
import numpy as np
from numpy.random import default_rng

from util.optimization_data import SingleRunData, PopulationData
from util.pop_diversity_metrics import PopDiversityMetric
from util.constants import RNG_SEED, DATASET_PATH, BATCH_SIZE, EPOCHS, POP_SIZE, MAX_ITER, NUM_RUNS

execute_training = False

In [ ]:
def optimization(algorithm: Algorithm, task: Task, single_run_data: SingleRunData):
    r"""An adaptation of NiaPy Algorithm run method.

    Args:
        algorithm (Algorithm): Algorithm.
        task (Task): Task with pre configured parameters.
        single_run_data (SingleRunData): Instance for archiving optimization results
    """
    try:
        algorithm.callbacks.before_run()
        algorithm.rng = default_rng(seed=RNG_SEED)
        pop, fpop, params = algorithm.init_population(task)
        # reset seed to random
        algorithm.rng = default_rng()
        xb, fxb = algorithm.get_best(pop, fpop)
        while not task.stopping_condition():
            # save population data
            pop_data = PopulationData(
                population=np.array(pop), population_fitness=np.array(fpop)
            )
            pop_data.calculate_metrics(
                [
                    PopDiversityMetric.PDC,
                    PopDiversityMetric.PED,
                    PopDiversityMetric.PMD,
                    PopDiversityMetric.AAD,
                    PopDiversityMetric.PDI,
                    PopDiversityMetric.PFSD,
                    PopDiversityMetric.PFMea,
                    PopDiversityMetric.PFMed,
                ],
                task.problem,
            )
            single_run_data.add_population(pop_data)
            algorithm.callbacks.before_iteration(pop, fpop, xb, fxb, **params)
            pop, fpop, xb, fxb, params = algorithm.run_iteration(
                task, pop, fpop, xb, fxb, **params
            )

            algorithm.callbacks.after_iteration(pop, fpop, xb, fxb, **params)
            task.next_iter()
        algorithm.callbacks.after_run()
        single_run_data.calculate_indiv_diversity_metrics()
        return xb, fxb * task.optimization_type.value
    except BaseException as e:
        if (
            threading.current_thread() is threading.main_thread()
            and multiprocessing.current_process().name == "MainProcess"
        ):
            raise e
        algorithm.exception = e
        return None, None

In [ ]:
def optimization_runner(
    algorithm: Algorithm, problem: Problem, max_iter: int, runs: int
):
    r"""An adaptation of NiaPy ALgorithm run method.

    Args:
        algorithm (Algorithm): Algorithm.
        problem (Problem): Optimization problem.
        max_iter (int): Optimization stopping condition.
        runs (int): Number of runs to execute.
    """
    for r in tqdm(range(runs)):
        task = Task(problem, max_iters=max_iter)

        single_run_data = SingleRunData(
            algorithm_name=algorithm.Name,
            algorithm_parameters=algorithm.get_parameters(),
            problem_name=problem.name(),
            max_iters=max_iter,
        )

        results = optimization(algorithm, task, single_run_data)
        single_run_data.export_to_json(
            os.path.join(
                DATASET_PATH, algorithm.Name[0], problem.name(), f"run_{r:05d}.json"
            )
        )

In [ ]:
for algorithm in [BatAlgorithm(population_size=POP_SIZE), FireflyAlgorithm(population_size=POP_SIZE), ParticleSwarmOptimization(population_size=POP_SIZE, c1=1.0, c2=1.0)]:
    problem = Schwefel()
    optimization_runner(algorithm, problem, MAX_ITER, NUM_RUNS)

### Metrics comparison

In [ ]:
for algorithm in os.listdir(DATASET_PATH):
    for problem in os.listdir(os.path.join(DATASET_PATH, algorithm)):
        runs = os.listdir(os.path.join(DATASET_PATH, algorithm, problem))
        run_path = os.path.join(DATASET_PATH, algorithm, problem, runs[0])
        pop_metrics = SingleRunData.import_from_json(run_path).get_pop_diversity_metrics_values(normalize=True)
        pop_metrics.plot(title=" ".join([algorithm, problem]), figsize=(20,5))

### ML Data generation

In [ ]:
class data_generator(torch.utils.data.Dataset):
    def __init__(self, data_path_list, classes) -> None:
        super().__init__()
        self.data_path_list = data_path_list
        self.classes = classes

    def __len__(self):
        return len(self.data_path_list)

    def __getitem__(self, index):
        run = SingleRunData.import_from_json(self.data_path_list[index])
        pop_metrics = run.get_pop_diversity_metrics_values(normalize=True).to_numpy()
        indiv_metrics = run.get_indiv_diversity_metrics_values(
            normalize=True
        ).to_numpy()

        pca = PCA(n_components=indiv_metrics.shape[1])
        principal_components = pca.fit_transform(indiv_metrics).flatten()
        variance = pca.explained_variance_ratio_

        return (
            torch.from_numpy(pop_metrics).float(),
            torch.from_numpy(principal_components).float(),
            torch.tensor(self.classes.index(run.algorithm_name[0])),
        )

### Model

In [ ]:
class LSTM(nn.Module):
    def __init__(self, input_dim, aux_input_dim, num_classes, hidden_dim=256, num_layers=3) -> None:
        super().__init__()

        self.lstm = nn.LSTM(
            input_size=input_dim,
            hidden_size=hidden_dim,
            num_layers=num_layers,
            batch_first=True,
            dropout=0.8
        )
        self.fc = nn.Linear(hidden_dim + aux_input_dim, num_classes)

    def forward(self, x, aux):
        lstm_out, (h_n, c_n) = self.lstm(x)
        features_0 = lstm_out[:, -1]
        features = torch.concat([features_0, aux], dim=1)
        out = self.fc(features)

        return features, out

### Data loading and preprocessing

In [ ]:
dataset_paths = []
classes = []
for algorithm in os.listdir(DATASET_PATH):
    classes.append(algorithm)
    for problem in os.listdir(os.path.join(DATASET_PATH, algorithm)):
        for run in os.listdir(os.path.join(DATASET_PATH, algorithm, problem)):
            dataset_paths.append(os.path.join(DATASET_PATH, algorithm, problem, run))

x_train, x_test = sklearn.model_selection.train_test_split(dataset_paths, test_size=0.2, shuffle=True, random_state=RNG_SEED)
x_train, x_val = sklearn.model_selection.train_test_split(x_train, test_size=0.25, shuffle=True, random_state=RNG_SEED)

In [ ]:
train_dataset = data_generator(x_train, classes)
train_data_loader = data.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, pin_memory=True, num_workers=os.cpu_count())

val_dataset = data_generator(x_val, classes)
val_data_loader = data.DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=True, pin_memory=True, num_workers=os.cpu_count())

test_dataset = data_generator(x_test, classes)
test_data_loader = data.DataLoader(test_dataset, batch_size=1, shuffle=True, pin_memory=True, num_workers=os.cpu_count())

pop_features, indiv_features, target = next(iter(train_data_loader))
print(np.shape(pop_features))
print(np.shape(indiv_features))
print(np.shape(target))

In [ ]:
def nn_train(model, train_data_loader, val_data_loader, epochs, loss_fn, optimizer, device, batch_size, model_file_name):
    loss_values = []
    val_loss_values = []

    for epoch in range(epochs):
        loss_sum = 0.0
        val_loss_sum = 0.0
            
        model.train()
        for batch in train_data_loader:
            pop_features, indiv_features, target = batch
            
            target = target.to(device)
            pop_features = pop_features.to(device)
            indiv_features = indiv_features.to(device)

            _, pred = model(pop_features, indiv_features)
            loss = loss_fn(pred, target)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            loss_sum += loss.item()

        model.eval()
        with torch.no_grad():
            for batch in val_data_loader:
                pop_features, indiv_features, target = batch
            
                target = target.to(device)
                pop_features = pop_features.to(device)
                indiv_features = indiv_features.to(device)

                _, pred = model(pop_features, indiv_features)
                loss = loss_fn(pred, target)

                val_loss_sum += loss.item()

        loss_values.append(loss_sum/batch_size)
        val_loss_values.append(val_loss_sum/batch_size)
        print(f"epoch: {epoch + 1}, loss: {loss_sum/batch_size :.10f}, val_loss: {val_loss_sum/batch_size :.10f}")

        torch.save(model, model_file_name)

    x = [*range(1, epochs+1)]
    plt.plot(x, loss_values, label="train loss")
    plt.plot(x, val_loss_values, label="val loss")
    plt.legend()
    plt.show()
    plt.savefig("loss_plot.png")

In [ ]:
def test_nn(model, test_data_loader, device, classes):
    model.eval()

    y_pred = []
    y_target = []

    for batch in test_data_loader:
        pop_features, indiv_features, target = batch
            
        target = target.to(device)
        pop_features = pop_features.to(device)
        indiv_features = indiv_features.to(device)


        with torch.no_grad():
            _, pred = model(pop_features, indiv_features)
            
            y_pred.append(torch.argmax(pred).cpu().numpy())
            y_target.append(target.cpu().numpy()[0])

    print(classification_report(y_target, y_pred))
    cf_matrix = confusion_matrix(y_target, y_pred)
    df_cm = pd.DataFrame(cf_matrix, index = [i for i in classes], columns = [i for i in classes])
    plt.figure(figsize = (12,7))
    sb.heatmap(df_cm, annot=True)

In [ ]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
print(device)
print("CPUs: ", os.cpu_count())

In [ ]:
pop_features, indiv_features, target = next(iter(train_data_loader))
model = LSTM(np.shape(pop_features)[2], np.shape(indiv_features)[1], len(classes))
optimizer = torch.optim.Adam(model.parameters(), lr=1e-5)
loss_fn = nn.CrossEntropyLoss()
model_file_name = f"lstm_model.pt"

if execute_training:
    model.to(device)
    nn_train(model, train_data_loader, val_data_loader, EPOCHS, loss_fn, optimizer, device, BATCH_SIZE, model_file_name)
    torch.save(model, model_file_name)
else:
    model = torch.load(model_file_name, map_location=torch.device(device))
    model.to(device)
    loss_plot = np.asarray(Image.open('loss_plot.png'))
    plt.axis("off")
    plt.imshow(loss_plot)

In [ ]:
test_nn(model, test_data_loader, device, classes)